# 将数据分组,并用软连接表示
- 原始数据 25000
- divide in to
- train
 - dogs[1000]
 - cats[1000]
- validation
 - dogs[400]
 - cats[400]

In [47]:
import os
from random import shuffle
import shutil
import numpy as np
from collections import defaultdict
#seed 
np.random.seed(2018)

source_dir = '/home/mxq/Jupyter/Keras/dogsvscats/data/train/'
#target_dir = '/home/mxq/Jupyter/Keras/dogsvscats/data/train_sym'
target_dir = '/home/mxq/Jupyter/Keras/dogsvscats/data/test_sym'
def datadivide():
    samples,classes = generate_valid_imagepaths(source_dir)
    for _ in classes.items():
        #shuffle(_[1])
        classes[_[0]] = _[1][2000:2500]
    dothings(classes)
    print('done')
def dothings(classes):
    for class_path in classes.items():
        sub_dir = os.path.join(target_dir,class_path[0])
        if os.path.exists(sub_dir):
            shutil.rmtree(sub_dir)
        os.mkdir(sub_dir)
        filepaths = class_path[1]
        symlink_img(filepaths,sub_dir)

def symlink_img(filepaths,target_dir):
    for source_filepath in filepaths:
        symfilepath = os.path.split(source_filepath)[-1]
        target_filepath = os.path.join(target_dir,symfilepath)
        os.symlink(source_filepath,target_filepath)
    
def generate_valid_imagepaths(directory,follow_links=False):
    
    def _recursive_list(subpath):
        return sorted(os.walk(subpath,followlinks=follow_links),key=lambda tpl:tpl[0])
    samples = 0
    classes_file = defaultdict(list)
    for dirpath, _, files in _recursive_list(directory):
        for fname in sorted(files,key=lambda s:int(s[4:-4])):
        #for fname in sorted(files):
            if fname[:3] == 'dog':
               filepath = os.path.join(dirpath, fname)
               classes_file['dogs'].append(filepath)
               samples += 1
            else:
                filepath = os.path.join(dirpath, fname)
                classes_file['cats'].append(filepath)
                samples += 1
                
    return samples,classes_file
if __name__ == '__main__':
    datadivide()
    

done


# Data Augmentation Test

In [4]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import shutil
datagen = ImageDataGenerator(rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')
img = load_img('./data/train_sym/cats/cat.1.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0

if os.path.exists('data/preview/'):
    shutil.rmtree('data/preview/')
os.mkdir('data/preview/')
for batch in datagen.flow(x,batch_size=1,save_to_dir='data/preview',save_prefix='cat',save_format='jpeg'):
    i += 1
    if i > 20:
        break
print('done')

done


# Small Convnet

In [176]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense

model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])



# 从文件夹中读取数据

In [177]:
## Let's prepare our data. We will use .flow_from_directory() to generate batches 
#of image data (and their labels) directly from our jpgs in their respective folders.
batch_size = 16
# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
# This is the augmentation configuration we will use for testing:
# Only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# This is a generator that will read pictures found in subfolder of 'data/train_sym',
# and indefinitely generate batches of augmented image data.

train_generator = train_datagen.flow_from_directory(
                  'data/train_sym/',
                   target_size=(150,150),
                   batch_size=batch_size,
                   class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
                   'data/validation_sym/',
                    target_size=(150,150),
                    batch_size=batch_size,
                    class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


# 训练模型

In [11]:
'''model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')'''

Epoch 1/50
125/125 [==============================] - 57s 453ms/step - loss: 0.7032 - acc: 0.5165 - val_loss: 0.6715 - val_acc: 0.5687
Epoch 2/50
125/125 [==============================] - 57s 457ms/step - loss: 0.6812 - acc: 0.5640 - val_loss: 0.6496 - val_acc: 0.6350
Epoch 3/50
125/125 [==============================] - 55s 440ms/step - loss: 0.6529 - acc: 0.6230 - val_loss: 0.6526 - val_acc: 0.5950
Epoch 4/50
125/125 [==============================] - 57s 453ms/step - loss: 0.6311 - acc: 0.6480 - val_loss: 0.5919 - val_acc: 0.6925
Epoch 5/50
125/125 [==============================] - 55s 443ms/step - loss: 0.6108 - acc: 0.6810 - val_loss: 0.5702 - val_acc: 0.6987
Epoch 6/50
125/125 [==============================] - 55s 438ms/step - loss: 0.5962 - acc: 0.7040 - val_loss: 0.5455 - val_acc: 0.7000
Epoch 7/50
125/125 [==============================] - 55s 439ms/step - loss: 0.5630 - acc: 0.7205 - val_loss: 0.5663 - val_acc: 0.6913
Epoch 8/50
125/125 [==============================] - 5

 # 模型预测

In [223]:
import os
import cv2
import numpy as np
import pandas as pd

#seed
np.random.seed(2018)
test_batch_size = 16
pre_gen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_generator = pre_gen.flow_from_directory('data/train_sym',(150,150),
                                         class_mode='binary',shuffle=False,batch_size=test_batch_size)
steps_test = test_generator.samples

model.load_weights('first_try.h5')
#pred = model.predict_generator(test_generator,steps=steps_test)
#output = open("pred.npy",'wb')
#np.save(output,pred)


Found 2000 images belonging to 2 classes.


In [117]:
output2 = open("pred.npy",'rb')
pred_load = np.load(output2)
#model.evaluate_generator(test_generator)
#from postprocess import postprocess
for x in pred_load:
    print(x)
    

[0.569524]
[0.24353099]
[0.6307373]
[2.4851664e-05]
[0.14670713]
[0.02440804]
[0.0080556]
[0.11119525]
[0.00990648]
[0.5032455]
[0.4362974]
[0.10433654]
[0.09692132]
[0.09082355]
[0.24296084]
[0.21188118]
[0.02284339]
[0.09276731]
[0.15496062]
[0.28689238]
[0.27981094]
[0.01421825]
[0.03546935]
[0.13426065]
[0.02428766]
[0.3039123]
[0.2389768]
[0.24949487]
[0.25882527]
[0.17203727]
[0.03869753]
[0.27117956]
[0.01065296]
[0.13077316]
[0.02419917]
[0.00784016]
[0.06829943]
[0.3648785]
[0.69841176]
[0.04894631]
[0.08862553]
[0.00248957]
[0.03965409]
[0.03877462]
[0.4040083]
[0.3375469]
[0.11871459]
[0.10982618]
[0.29416734]
[0.24238849]
[0.12542868]
[0.4763415]
[0.23393305]
[0.27782378]
[0.08377693]
[0.1516005]
[0.2945772]
[0.10152912]
[0.07677276]
[0.17758442]
[0.09594364]
[0.7177048]
[0.00453952]
[0.9781145]
[0.00463718]
[0.18255132]
[0.1546647]
[0.15818666]
[0.00682027]
[0.09713107]
[0.21686982]
[0.53013295]
[0.23304376]
[0.0779893]
[0.38681173]
[0.22856914]
[0.03384158]
[0.15612671]
[

In [179]:
def find_class_index(a):
    b = {}
    for i in range(len(a)):
        if a[i] > .5:
            b[i] = 1
        else:
            b[i] = 0
    return b
def find_class_pred_index(a):
    b = {}
    for i in range(len(a)):
        if a[i][0] > .5:
            b[i] = 1
        else:
            b[i] = 0
    return b

In [226]:
drink_classes = {0:'cat',1:'dog'}
test_batch_size = 16
epoch = test_generator.samples//test_batch_size+1
def check_wrong_picture():
    wrong_pictures = []
    for i in range(epoch):
        x = test_generator.next()
        
        true_y = x[1]
        predict = model.predict_on_batch(x[0])
        true_class = find_class_index(true_y)

        predict_class = find_class_pred_index(predict)
        for j in range(len(x[0])):
            true_class[j] = drink_classes[true_class[j]]
            predict_class[j] = drink_classes[predict_class[j]]
            
            if not predict_class[j] == true_class[j]:
                #print(true_y[j],predict[j])
                wrong_picture = []
                wrong_picture.append(predict_class[j])
                wrong_picture.append(true_class[j])
                wrong_pictures.append(wrong_picture)
    return wrong_pictures             
wrong_pictures = check_wrong_picture()
print(len(wrong_pictures))

274
